In [1]:
import os
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: True


In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [3]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/sb_cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [4]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    action_training_input = data['action_training_input']
    recurrent_training_input = data['recurrent_training_input']
    prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    training_target_sequence = data['training_target_sequence']
    action_testing_input = data['action_testing_input']
    recurrent_testing_input = data['recurrent_testing_input']
    prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()

    # Create model
    
    # Action Model
    action_input_layer = Input(shape=action_training_input[0].shape, name='action')
    action_model = Dropout(rate=0.5)(action_input_layer)
    action_model = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
    
    # Recurrent Input
    recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
    recurrent_model = Dropout(rate=0.5)(recurrent_model)
    recurrent_model = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
    
    # Prior Input
    prior_input_layer = Input(shape=prior_training_input[0].shape, name='prior')
    prior_model = Dropout(rate=0.5)(prior_input_layer)
    prior_model = Dense(units=64, activation='sigmoid')(prior_model)
    
    # Combined Model
    model = Concatenate()([action_model, recurrent_model, prior_model])
    model = Dropout(rate=0.5)(model)
    
    # Combined Output
    completion_output_layer = Dense(units=1, activation='sigmoid', name='completion')(model)
    problems_output_layer = Dense(units=1, activation='linear', name='problems')(model)
    
    combined_model = Model([action_input_layer, recurrent_input_layer, prior_input_layer], [completion_output_layer, problems_output_layer])
    combined_model.compile(optimizer='adam', loss={'completion': 'binary_crossentropy', 'problems': 'mse'})

    # Train model
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    weights = {'completion': np.ones_like(completion_training_target) * 16, 'problems': completion_training_target}
    combined_model.fit(x={'action': action_training_input, 'recurrent': recurrent_training_input, 'prior': prior_training_input},
                       y={'completion': completion_training_target, 'problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'action': action_testing_input, 
                                                                                 'recurrent': recurrent_testing_input, 
                                                                                 'prior': prior_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    

pd.concat(results).to_csv('merged_cross_validation_results.csv', index=False)

Epoch 1/1000
162/162 [==============================] - 29s 131ms/step - loss: 38.4142 - completion_loss: 9.1016 - problems_loss: 29.3126 - val_loss: 16.4898 - val_completion_loss: 9.1466 - val_problems_loss: 7.3433
Epoch 2/1000
162/162 [==============================] - 20s 126ms/step - loss: 33.7866 - completion_loss: 8.3722 - problems_loss: 25.4144 - val_loss: 16.2273 - val_completion_loss: 9.1357 - val_problems_loss: 7.0916
Epoch 3/1000
162/162 [==============================] - 23s 145ms/step - loss: 32.5190 - completion_loss: 8.1544 - problems_loss: 24.3646 - val_loss: 16.1147 - val_completion_loss: 8.9706 - val_problems_loss: 7.1441
Epoch 4/1000
162/162 [==============================] - 19s 117ms/step - loss: 31.8787 - completion_loss: 7.9922 - problems_loss: 23.8865 - val_loss: 16.4933 - val_completion_loss: 9.2567 - val_problems_loss: 7.2366
Epoch 5/1000
162/162 [==============================] - 19s 120ms/step - loss: 31.3266 - completion_loss: 7.8699 - problems_loss: 23.456

Epoch 14/1000
162/162 [==============================] - 16s 99ms/step - loss: 15.1402 - completion_loss: 7.7789 - problems_loss: 7.3613 - val_loss: 18.3466 - val_completion_loss: 11.5138 - val_problems_loss: 6.8328
Epoch 1/1000
162/162 [==============================] - 23s 97ms/step - loss: 100.3279 - completion_loss: 9.4388 - problems_loss: 90.8891 - val_loss: 15.7328 - val_completion_loss: 9.3070 - val_problems_loss: 6.4258
Epoch 2/1000
162/162 [==============================] - 14s 87ms/step - loss: 35.3941 - completion_loss: 8.5733 - problems_loss: 26.8207 - val_loss: 15.2214 - val_completion_loss: 9.2593 - val_problems_loss: 5.9621
Epoch 3/1000
162/162 [==============================] - 13s 81ms/step - loss: 34.2869 - completion_loss: 8.3211 - problems_loss: 25.9659 - val_loss: 15.2543 - val_completion_loss: 9.3150 - val_problems_loss: 5.9393
Epoch 4/1000
162/162 [==============================] - 13s 83ms/step - loss: 33.4503 - completion_loss: 8.1847 - problems_loss: 25.2655 -

In [5]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('merged_cross_validation_results.csv')

In [6]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []

# Calculate the metrics for each sequence
for sequence, df in results.groupby('target_sequence'):
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for fold, df in results.groupby('fold'):
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
completion_target = results['completion_target']
completion_prediction = results['completion_prediction']
completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_mse = mean_squared_error(completion_target, completion_prediction)

problems_target = results[results['completion_target'] == 1]['problems_target']
problems_prediction = results[results['completion_target'] == 1]['problems_prediction']
problems_mae = mean_absolute_error(problems_target, problems_prediction)
problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_mse = mean_squared_error(problems_target, problems_prediction)

metrics.append(['all_target_data', 
                'None', 
                len(results), 
                completion_auc, 
                completion_acc, 
                completion_r2, 
                completion_ev, 
                completion_mse, 
                problems_mae, 
                problems_mape, 
                problems_r2, 
                problems_ev, 
                problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['sequence_id', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('merged_cross_validation_metrics.csv', index=False)

In [7]:
metrics.iloc[-6:]

,sequence_id,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
26,fold_0,None,1722,0.722445,0.704413,0.136081,0.136546,0.186887,2.457606,0.213971,0.057924,0.064409,14.747056
27,fold_1,None,1722,0.718018,0.746225,0.115982,0.121512,0.174254,2.199983,0.359514,-0.068016,-0.012085,7.948823
28,fold_2,None,1721,0.773003,0.771644,0.174369,0.175858,0.154986,3.080280,0.207400,-0.029412,-0.016144,83.572682
29,fold_3,None,1721,0.731475,0.757118,0.134823,0.142114,0.171979,2.241908,0.266352,0.042353,0.043226,11.474542
30,fold_4,None,1721,0.711921,0.747240,0.127299,0.127352,0.176408,2.087260,0.264356,0.011434,0.016705,8.411141
31,all_target_data,None,8607,0.730210,0.745324,0.139281,0.139349,0.172904,2.417625,0.262638,-0.003937,-0.003671,25.745528
